In [40]:
import warnings
import csv
import math
import time
import random
from time import time
import hashlib
import psutil
import GPUtil
import numpy as np
import networkx as nx
from itertools import product
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import rustworkx as rx
from rustworkx.visualization import mpl_draw as draw_graph
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

from qiskit import QuantumCircuit, transpile, assemble
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import QAOAAnsatz
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import Session, EstimatorV2 as Estimator
from qiskit_ibm_runtime import SamplerV2 as Sampler
from qiskit.providers.fake_provider import GenericBackendV2
from qiskit_aer import AerSimulator

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import random_split
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool, BatchNorm, SAGPooling
import torch.nn as nn


In [41]:
def calculate_max_cut(bitstring, graph):
    cut_value = 0
    for (u, v) in graph.edges():
        if bitstring[u] != bitstring[v]:  # Different partitions
            cut_value += graph[u][v]['weight']
    return cut_value

# Function to create QAOA circuit
from qiskit import QuantumCircuit

def create_qaoa_circuit(gamma, beta, graph, depth):
    num_qubits = len(graph.nodes)
    qc = QuantumCircuit(num_qubits, num_qubits)  # Allocate classical bits

    # Apply Hadamard gates to all qubits
    qc.h(range(num_qubits))
    qc.barrier()

    for _ in range(depth):
        # Apply the problem unitary (Cost Hamiltonian)
        for (u, v) in graph.edges:
            qc.cx(u, v)
            qc.rz(2 * gamma * graph[u][v]['weight'], v)
            qc.cx(u, v)
        qc.barrier()

        # Apply the mixer unitary (Mixer Hamiltonian)
        for qubit in range(num_qubits):
            qc.rx(2 * beta, qubit)
        qc.barrier()

    # Measurement
    qc.measure(range(num_qubits), range(num_qubits))

    return qc


In [42]:
def QAOA(
    beta: float,
    gamma: float,
    fakeBackend: bool,
    depth: int,
    adjacencyList: list[list[int]],
) -> dict:
    if fakeBackend:
        backend = GenericBackendV2(len(adjacencyList))
    else:
        backend = rensslearBackend  # Define your actual backend here
    qc = QAOA_circuit(beta, gamma, depth, adjacencyList)
    job = backend.run(qc, shots=1024)
    result = job.result()
    return result.get_counts()


# Takes params->[beta, gamma], a bool representing if the backend is fake, depth of the circuit, and adjacnecy list;
# returns the average cut of the given graph
def cost_function(params, fakeBackend: bool, depth: int, adj_list: list[list[int]]):
    beta, gamma = params
    counts = QAOA(beta, gamma, fakeBackend, depth, adj_list)
    # Calculate the expectation value of the cost Hamiltonian
    cost = 0
    for bitstring, count in counts.items():
        bit_val = [int(bit) for bit in bitstring]
        cut_value = 0
        for i in range(len(adj_list)):
            for j in adj_list[i]:
                if i < j:
                    cut_value += bit_val[i] != bit_val[j]
        cost += cut_value * count
    return cost / 1024
def convert_to_graph_dict(G):
    graph_dict = {}

    for u, neighbors in enumerate(G):
        for v in neighbors:
            if (v, u) not in graph_dict:  # Avoid duplicate edges for undirected graph
                graph_dict[(u, v)] = 1.0


    return graph_dict

In [43]:
# Takes a beta, gamma, depth, and adjacencyList; returns the QAOA circuit
def QAOA_circuit(beta: float, gamma: float, depth: int, adjacencyList: list[list[int]]):
    num_qubits = len(adjacencyList)
    qc = QuantumCircuit(num_qubits, num_qubits)  # Allocate classical bits

    # Initial Hadamard gates
    qc.h(range(num_qubits))
    qc.barrier()

    for _ in range(depth):
        # Apply the Hamiltonian cost function
        for edgeIndex, neighbors in enumerate(adjacencyList):
            for edge in neighbors:
                if edge > edgeIndex:
                    qc.cz(edgeIndex, edge)
                    qc.rz(2 * gamma, edgeIndex)
                    qc.cz(edge, edgeIndex)
        qc.barrier()

        # Apply RX gates to each qubit for the mixer Hamiltonian
        for qubit in range(num_qubits):
            qc.rx(2 * beta, qubit)
        qc.barrier()

    # Measurement
    qc.measure(range(num_qubits), range(num_qubits))

    return qc

    # Takes a beta, gamma, bool representing if the backend is fake, depth of the circuit, and adjacnecy list; returns a dict of the bitstrings produced
def QAOA(
    beta: float,
    gamma: float,
    fakeBackend: bool,
    depth: int,
    adjacencyList: list[list[int]],
) -> dict:
    if fakeBackend:
        backend = GenericBackendV2(len(adjacencyList))
    else:
        backend = rensslearBackend  # Define your actual backend here
    qc = QAOA_circuit(beta, gamma, depth, adjacencyList)
    job = backend.run(qc, shots=1024)
    result = job.result()
    return result.get_counts()


# Takes params->[beta, gamma], a bool representing if the backend is fake, depth of the circuit, and adjacnecy list;
# returns the average cut of the given graph
def cost_function(params, fakeBackend: bool, depth: int, adj_list: list[list[int]]):
    beta, gamma = params
    counts = QAOA(beta, gamma, fakeBackend, depth, adj_list)
    # Calculate the expectation value of the cost Hamiltonian
    cost = 0
    for bitstring, count in counts.items():
        bit_val = [int(bit) for bit in bitstring]
        cut_value = 0
        for i in range(len(adj_list)):
            for j in adj_list[i]:
                if i < j:
                    cut_value += bit_val[i] != bit_val[j]
        cost += cut_value * count
    return cost / 1024

In [44]:
def read_csv(filename):
    with open(filename, mode='r', newline='') as file:
        reader = csv.reader(file)
        for row in reader:
            print(row)

In [45]:
def create_adjacency_matrix(adj_list, num_nodes):
    adj_matrix = [[0] * num_nodes for _ in range(num_nodes)]  # Initialize with zeros
    for node, neighbors in enumerate(adj_list):
        for neighbor in neighbors:
            adj_matrix[node][neighbor] = 1
            adj_matrix[neighbor][node] = 1  # Assuming undirected graph
    return adj_matrix

In [46]:
def adjacency_matrix_to_list(adj_matrix):
    adjacency_list = []
    num_nodes = len(adj_matrix)

    for i in range(num_nodes):
        neighbors = []
        for j in range(num_nodes):
            if adj_matrix[i][j] != 0:
                neighbors.append(j)
        adjacency_list.append(neighbors)

    return adjacency_list

In [47]:
def createTorchObject(graph):
    return torch.tensor(graph)
    
class DataObject:
    def __init__(self, gamma, beta, graph):
        self.gamma = gamma
        self.beta = beta
        self.graph = graph
        self.torchObject = createTorchObject(graph)
        
    def getGamma(self):
        return self.gamma
        
    def getBeta(self):
        return self.beta
        
    def getGraph(self):
        return self.graph       
    
class Dataset(torch.utils.data.Dataset):
    def __init__(self, list_IDs, dataObjects):
        self.list_IDs = list_IDs # list
        self.dataObjects = dataObjects # dictionary
        
    def __len__(self):
            return len(self.list_IDs)
        
    def __getitem__(self, index):
        ID = self.list_IDs[index]
        dataObject = self.dataObjects[ID]
        X = dataObject.torchObject 
        Y = [dataObject.getGamma(), dataObject.getBeta()]
        return X, Y

In [48]:
def all_csv_parser(filename, skip):
  data = []
  IDCounter = 0
  list_IDs = []
  dataObjects = dict()
  with open(filename, mode='r', newline='') as file:
      # Make an array like this: [[graphid, adjlist], [graphid, adjlist], ...]
      reader = csv.reader(file)
      max_length = 0
      for row in reader:
          data.append(row)
      # Remove the header
      data = data[1:]
      new_data = []
      for row in data:
        if type(eval(row[1])) != list:
          continue
        new_data.append(row)
        if len(eval(row[1])) > max_length:
          max_length = len(eval(row[1]))
      data = new_data
      # Convert the graph[n][1] from a string into an adjacency list
      for i in range(skip, len(data)):
          currentId = 'id-' + str(IDCounter)
          list_IDs.append(currentId)
          beta = float(data[i][2])
          gamma = float(data[i][3])
          graph = eval(data[i][1])
          matrix = create_adjacency_matrix(graph, max_length)
          dataObjects[currentId] = DataObject(gamma, beta, matrix)
          IDCounter += 1
  print("length: ", IDCounter)
  return list_IDs, dataObjects, max_length

In [49]:
# fullDataset = Dataset(list_IDs, dataObjects)
# Read graphs from csv
num = 3
filename = ''
if num == 1:
    filename = './allCSVFiles-6.csv'
elif num == 2:
    filename = "./allCSVFiles-3.csv"
elif num == 3:
    filename = "dataset-1000.csv"
    
# convert to list of lists
list_IDs, dataObjects, max_length = all_csv_parser(filename, 500)

# convert to classes
train_IDs, test_IDs = train_test_split(list_IDs, test_size=0.3, random_state=42)
val_IDs, test_IDs = train_test_split(test_IDs, test_size=0.5, random_state=42)

# Create datasets split the data
train_dataset = Dataset(train_IDs, dataObjects)
val_dataset = Dataset(val_IDs, dataObjects)
test_dataset = Dataset(test_IDs, dataObjects)


length:  498


In [50]:
print(len((dataObjects['id-0'].getGraph()[0])))

24


In [51]:
# Come back to this should the gamma and beta be tensors?
# should the x object be returned back with the number one for model creation?
# print(fullDataset.__getitem__(0))

In [52]:
def custom_loss(actual_params, predicted_params, graphs, batch_size):
    if len(actual_params) < batch_size:
        print(f"Skipping batch as its size {len(actual_params)} is smaller than {batch_size}.")
        return torch.tensor(0.0, device='cuda', requires_grad=True)
    
    randomIndicies = random.sample(range(0, batch_size), 5)
    totalLoss = 0
    for i in randomIndicies:
        gammaPredicted = predicted_params[0][i].item()
        betaPredicted = predicted_params[1][i].item()
        gammaKnown = actual_params[i][0].item()
        betaKnow = actual_params[i][1].item()
        graph = graphs[i].tolist()
        expected_cost = cost_function([betaKnow, gammaKnown], True, 1, graph)
        predicted_cost = cost_function([betaPredicted, gammaPredicted], True, 1, graph)
        totalLoss += (predicted_cost - expected_cost)
    #print("predicted", predicted_params)
    print("total Loss: ", totalLoss)
    totalLoss += 10
    if totalLoss < 0:
        totalLoss = 0.0
    return torch.tensor(totalLoss, device='cuda', requires_grad=True)


In [53]:
def adj_to_edge_index(adj):
    edge_index = adj.nonzero(as_tuple=True)
    edge_index = torch.stack(edge_index, dim=0)
    return edge_index

In [54]:
class GNNRegressor(nn.Module):
    def __init__(self):
        super(GNNRegressor, self).__init__()
        self.conv1 = GCNConv(in_channels=1, out_channels=128)
        self.conv2 = GCNConv(in_channels=128, out_channels=256)
        self.conv3 = GCNConv(in_channels=256, out_channels=256)
        self.conv4 = GCNConv(in_channels=256, out_channels=512)
        self.bn1 = nn.BatchNorm1d(128)
        self.bn2 = nn.BatchNorm1d(256)
        self.bn3 = nn.BatchNorm1d(256)
        self.bn4 = nn.BatchNorm1d(512)
        self.fc1 = nn.Linear(512, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 2)  # Predicting gamma and beta

    def forward(self, batch):
        adj_matrices, params = batch
        batch_size, num_nodes, _ = adj_matrices.size()
        
        outputs = []
        for i in range(batch_size):
            adj_matrix = adj_matrices[i]
            edge_index = adj_to_edge_index(adj_matrix)
            node_features = torch.ones((num_nodes, 1)).to(device)  # Example with single feature
            
            x = F.relu(self.bn1(self.conv1(node_features, edge_index)))
            x = F.relu(self.bn2(self.conv2(x, edge_index)))
            x = F.relu(self.bn3(self.conv3(x, edge_index)))
            x = F.relu(self.bn4(self.conv4(x, edge_index)))
            x = F.dropout(x, p=0.5, training=self.training)
            x = F.relu(self.fc1(x))
            x = F.relu(self.fc2(x))
            out = self.fc3(x.mean(dim=0))  # Assuming global pooling and reducing to (batch_size, 2)
            outputs.append(out)
        
        return torch.stack(outputs)
    #def forward(self, x):
      #  x = self.relu(self.conv1(x))
     #   x = self.pool(x)
      #  x = self.relu(self.conv2(x))
     #   x = self.pool(x)
     #   x = x.view(x.size(0), -1)  # Flatten the tensor
      #  x = self.relu(self.fc1(x))
      #  x = self.fc2(x)
      #  return x

In [38]:
use_cuda = torch.cuda.is_available()
if not use_cuda:
    print("not using GPU")
# might need to change to use multiple gpus
device = torch.device("cuda:3" if use_cuda else "cpu")
torch.backends.cudnn.benchmark = True

# load the data
current_batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=current_batch_size, shuffle=True, num_workers=8)
val_loader = DataLoader(val_dataset, batch_size=current_batch_size, shuffle=True, num_workers=8)
test_loader = DataLoader(test_dataset, batch_size=current_batch_size, shuffle=False, num_workers=8)

model = GNNRegressor().to(device)
max_epochs = 20
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=0.01)

In [35]:

# criterion = nn.MSELoss()
# scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5)


train_losses = []
validation_losses = []
for epoch in range(max_epochs):
    startTime = time()
    train_loss = 0.0  # Initialize train_loss for the epoch
    model.train()
    for batch in train_loader:
        # print(batch)
        # Unpack the batch into input features (X) and target labels (Y)
        X, Y = batch
        X = X.to(device)
        Y = (Y[0].to(device), Y[1].to(device))
        batch = [X, Y]
        optimizer.zero_grad()
        out = model(batch)
        predicted_params = out.view(-1, 2)  # Assuming your model output is of shape [batch_size, 2]
        if len(Y) == 3:
            print(Y)
            continue
        #actual_params = Y.view(-1, 2)  # Reshape Y to match the output
        # print(X.shape)
        batch_loss = custom_loss(predicted_params, Y, X, 32)
        batch_loss.backward()
        optimizer.step()
        train_loss += batch_loss.item()
    train_loss /= len(train_loader)
    train_losses.append(train_loss)
    print("epoch: ", epoch)
    print("Time ellapsed: ", startTime - time())

total Loss:  -0.755859375
total Loss:  -0.22265625
total Loss:  -0.0029296875
total Loss:  0.0732421875
total Loss:  0.40234375
total Loss:  -0.5009765625
total Loss:  0.4150390625
total Loss:  -0.0947265625
total Loss:  -0.29296875
total Loss:  -0.080078125
Skipping batch as its size 28 is smaller than 32.
epoch:  0
Time ellapsed:  -754.1124737262726
total Loss:  -0.3662109375
total Loss:  0.29296875
total Loss:  -0.611328125
total Loss:  -1.2421875
total Loss:  0.1201171875
total Loss:  0.2373046875
total Loss:  -0.40625
total Loss:  -1.4833984375
total Loss:  -0.0654296875
total Loss:  -0.1337890625
Skipping batch as its size 28 is smaller than 32.
epoch:  1
Time ellapsed:  -724.1959979534149
total Loss:  -0.017578125
total Loss:  -0.4580078125
total Loss:  -0.2890625
total Loss:  -0.3818359375
total Loss:  0.224609375
total Loss:  -0.1259765625
total Loss:  -0.021484375
total Loss:  0.193359375
total Loss:  0.56640625
total Loss:  -0.3603515625
Skipping batch as its size 28 is smal

In [36]:
torch.save(model.state_dict(), 'model_8_16_.pth')

In [37]:
checkpoint_dir = "checkpoints"
os.makedirs(checkpoint_dir, exist_ok=True)

checkpoint_path = os.path.join(checkpoint_dir, 'modelwithloss_8_16_.pt')
torch.save(model.state_dict(), checkpoint_path)
print(f"Model checkpoint saved to {checkpoint_path}")


NameError: name 'os' is not defined

In [55]:
use_cuda = torch.cuda.is_available()
if not use_cuda:
    print("not using GPU")
# might need to change to use multiple gpus
device = torch.device("cuda:3" if use_cuda else "cpu")
torch.backends.cudnn.benchmark = True
model = GNNRegressor().to(device)

In [56]:
model.load_state_dict(torch.load('model_8_16.pth'))
print("Model has been loaded and is in evaluation mode.")



Model has been loaded and is in evaluation mode.


/tmp/ipykernel_69400/3561937107.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('model_8_16.pth'))


In [ ]:
#RUN THIS
validation_losses = []
best_val_loss = float('inf')  # Initialize the best validation loss to infinity

for epoch in range(max_epochs):
    start_time = time.time()
    val_loss = 0.0  # Initialize val_loss for the epoch
    model.eval()
    
    with torch.no_grad():  # Disable gradient calculation for validation
        for batch in val_loader:
            # Unpack the batch into input features (X) and target labels (Y)
            X, Y = batch
            X = X.to(device)
            Y = (Y[0].to(device), Y[1].to(device))
            batch = [X, Y]
            
            # Forward pass
            out = model(batch)
            predicted_params = out.view(-1, 2)  # Assuming your model output is of shape [batch_size, 2]
            
            # Calculate loss
            batch_loss = custom_loss(predicted_params, Y, X, 32)
            val_loss += batch_loss.item()
    
    val_loss /= len(val_loader)
    validation_losses.append(val_loss)
    
    print(f"Epoch: {epoch+1}, Validation Loss: {val_loss:.4f}")
    print(f"Time elapsed: {time.time() - start_time:.2f} seconds")
    
    # Save the model if the validation loss is the best we've seen so far
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), f'model_epoch_{epoch+1}.pth')
        print(f"Model saved with validation loss {val_loss:.4f} at epoch {epoch+1}")

In [28]:
print(train_losses)

NameError: name 'train_losses' is not defined

In [57]:
graph1 = [[1, 12, 13, 8, 6, 4, 3], [11, 4, 9, 10, 0, 2, 3, 14, 12, 5], [8, 9, 1, 12, 10, 6, 14, 13], [14, 5, 12, 1, 10, 0], [10, 6, 1, 9, 5, 0], [3, 6, 8, 10, 14, 7, 4, 1, 12], [4, 5, 7, 13, 0, 2, 14], [9, 5, 6, 11], [5, 11, 2, 0, 12, 9], [1, 4, 7, 2, 14, 11, 13, 12, 8], [4, 1, 5, 3, 13, 2, 14], [1, 8, 14, 9, 13, 7, 12], [3, 13, 0, 2, 8, 1, 11, 9, 14, 5], [12, 0, 6, 14, 10, 9, 11, 2], [3, 11, 5, 9, 1, 13, 10, 2, 12, 6]]
graph2 = [[12, 16, 19, 11, 2, 1], [18, 15, 10, 12, 16, 4, 17, 0], [4, 19, 7, 9, 3, 0], [16, 7, 11, 15, 4, 8, 6, 2, 10, 18, 9], [2, 11, 5, 3, 8, 19, 1, 18, 13], [14, 13, 18, 19, 12, 8, 4, 11, 7, 6], [7, 15, 10, 3, 8, 5, 16], [6, 16, 2, 3, 14, 15, 8, 5], [12, 14, 17, 19, 5, 3, 6, 4, 18, 16, 7], [11, 10, 15, 12, 16, 2, 14, 3, 19], [9, 1, 6, 15, 3, 14], [9, 15, 3, 4, 14, 5, 12, 0], [8, 14, 1, 5, 9, 0, 11], [5, 19, 18, 15, 4], [12, 5, 8, 7, 15, 11, 9, 10, 17], [6, 11, 1, 9, 3, 14, 10, 7, 18, 13], [17, 7, 3, 9, 1, 0, 8, 6], [16, 8, 1, 14], [1, 5, 13, 8, 15, 3, 4], [2, 13, 5, 8, 4, 0, 9]]
graph3 = [[6, 7, 3, 12, 9, 10], [9, 11, 8, 12], [12, 8], [0, 9, 4], [6, 3, 9], [8, 10, 7], [0, 7, 4, 9], [0, 6, 5, 11], [5, 1, 2], [1, 3, 0, 6, 4], [11, 5, 0], [1, 10, 12, 7], [1, 0, 2, 11]]
graph4 = [[11, 12, 6, 3], [3, 10, 4, 11], [8, 11, 13, 14, 12, 6], [11, 13, 1, 0], [12, 5, 9, 14, 1], [12, 10, 9, 4, 7], [8, 0, 2], [14, 5], [2, 6, 9, 14], [12, 5, 8, 4, 10, 13], [5, 9, 1, 13], [3, 2, 0, 13, 1, 14], [5, 9, 0, 4, 2], [3, 2, 11, 9, 10], [7, 8, 2, 4, 11]]
graph5 = [[4, 8, 9, 1], [9, 0, 5], [5, 8, 3, 6], [2], [8, 0, 5, 9], [2, 4, 7, 1], [10, 2], [5], [2, 4, 0, 10, 9], [10, 1, 4, 0, 8], [9, 6, 8]]
graph6 = [[13, 9, 7, 15, 6, 11, 2, 16, 1, 3, 4, 12], [12, 14, 4, 8, 10, 0, 17, 18, 3], [19, 14, 13, 9, 15, 11, 17, 7, 18, 10, 0, 16, 3], [11, 18, 17, 16, 15, 12, 19, 4, 10, 2, 0, 1], [7, 12, 11, 8, 17, 6, 1, 16, 13, 19, 3, 18, 0, 10], [9, 19, 6, 11, 14, 13, 17], [16, 19, 5, 4, 8, 12, 0, 17, 15], [4, 14, 9, 11, 8, 2, 0, 18, 12, 10, 19], [4, 14, 6, 7, 17, 1, 11, 9, 15, 16, 10], [5, 7, 17, 12, 0, 2, 16, 13, 15, 8], [16, 12, 17, 15, 2, 1, 3, 7, 8, 4], [4, 3, 15, 14, 5, 7, 2, 13, 18, 16, 0, 8], [4, 10, 1, 9, 16, 6, 3, 13, 14, 17, 7, 0], [0, 18, 15, 19, 2, 12, 9, 11, 4, 5, 17, 14, 16], [7, 17, 8, 16, 11, 2, 15, 19, 1, 5, 12, 13, 18], [11, 13, 17, 14, 18, 3, 2, 16, 10, 0, 9, 6, 8], [18, 10, 6, 14, 3, 12, 15, 4, 9, 11, 19, 0, 2, 8, 13], [14, 3, 4, 9, 15, 10, 2, 8, 6, 12, 13, 5, 1], [16, 13, 3, 15, 2, 11, 7, 1, 4, 14], [6, 2, 5, 13, 14, 3, 16, 4, 7]]
graph7 = [[6, 1, 22], [20, 13, 0, 11, 6, 10, 3], [13, 18, 8, 9, 3, 17, 7, 14, 5, 16], [23, 13, 2, 1, 16], [5, 6, 13, 15, 19], [4, 16, 2, 11], [0, 15, 4, 1, 14, 10, 16, 12], [8, 18, 2, 22, 13], [10, 21, 7, 2, 19, 18, 11], [11, 2], [8, 23, 16, 11, 21, 18, 19, 1, 6], [1, 10, 9, 8, 5], [21, 17, 6], [2, 1, 19, 3, 4, 7, 15], [23, 6, 18, 22, 2], [22, 18, 6, 4, 17, 13], [10, 5, 6, 3, 2], [22, 12, 2, 15, 19], [15, 10, 21, 7, 2, 14, 23, 8], [13, 10, 21, 8, 4, 17], [1, 21], [10, 8, 12, 20, 18, 19, 22], [15, 17, 21, 14, 0, 7, 23], [14, 10, 3, 18, 22]]
graph8 = [[18, 3, 17, 14, 11, 15, 7, 5, 8, 4, 19], [14, 19, 15, 5, 9, 17, 18, 13, 12, 16, 7, 3], [18, 14, 12, 3, 19, 15, 13, 9, 16, 10, 7, 4], [17, 16, 0, 14, 15, 13, 2, 12, 9, 6, 20, 5, 1], [15, 10, 6, 17, 9, 8, 0, 11, 12, 13, 14, 19, 2], [11, 1, 15, 12, 14, 18, 0, 7, 3, 19, 8, 17, 20], [19, 16, 4, 20, 12, 13, 3, 7, 17, 14], [13, 12, 20, 0, 16, 18, 5, 14, 6, 11, 2, 1, 17, 15, 19, 10], [14, 13, 17, 12, 4, 9, 0, 11, 16, 19, 15, 20, 5], [1, 11, 4, 3, 2, 8, 16, 12, 19, 18], [14, 4, 18, 13, 11, 12, 2, 19, 7], [17, 13, 5, 9, 0, 19, 18, 8, 4, 10, 7, 12, 14, 15], [7, 8, 2, 5, 6, 3, 17, 1, 19, 9, 14, 4, 10, 15, 18, 11], [7, 11, 14, 8, 3, 1, 2, 6, 10, 4, 17, 18, 19], [1, 17, 18, 8, 10, 3, 13, 16, 2, 0, 5, 7, 12, 6, 4, 11], [4, 20, 1, 19, 17, 3, 5, 2, 0, 8, 12, 7, 11], [3, 6, 14, 7, 2, 9, 8, 1], [3, 11, 14, 15, 0, 8, 1, 4, 12, 6, 7, 13, 5], [0, 14, 2, 20, 10, 19, 1, 11, 5, 7, 12, 13, 9], [1, 6, 15, 18, 2, 11, 12, 8, 10, 9, 5, 13, 4, 0, 7], [15, 18, 6, 7, 3, 8, 5]]
graph9 = [[5, 2, 6, 1], [9, 2, 8, 0, 4], [0, 6, 1, 11, 9], [4, 9, 7], [3, 7, 10, 11, 1], [0, 10, 7, 11, 9], [10, 2, 0, 7, 9], [8, 4, 10, 5, 6, 3], [10, 7, 11, 1], [11, 1, 10, 5, 3, 2, 6], [5, 8, 6, 4, 7, 9], [9, 4, 8, 5, 2]]
graph10 = [[17, 1, 7, 2, 12, 6, 4, 9, 18, 10, 5], [0, 17, 2, 12, 14, 4], [8, 23, 0, 1, 21, 16, 10, 7, 6], [22, 6, 15, 18, 11, 5, 17, 21, 23, 10, 4, 9, 16, 8, 20], [16, 15, 0, 9, 12, 17, 11, 3, 22, 7, 18, 1], [20, 7, 9, 15, 3, 8, 19, 23, 18, 0], [3, 19, 20, 12, 0, 23, 15, 2, 17], [16, 5, 0, 22, 9, 15, 14, 8, 2, 11, 4, 17, 20], [12, 14, 2, 11, 21, 7, 22, 19, 18, 5, 16, 20, 9, 3, 17], [7, 5, 15, 17, 4, 11, 3, 0, 8, 12], [16, 14, 13, 21, 2, 3, 23, 20, 0], [22, 8, 17, 20, 19, 3, 9, 4, 18, 16, 7, 14], [8, 20, 17, 0, 6, 21, 4, 1, 22, 9, 15], [15, 21, 14, 10], [8, 20, 7, 13, 18, 10, 15, 16, 23, 1, 11, 17], [13, 3, 19, 20, 7, 4, 5, 9, 14, 6, 12], [4, 7, 10, 23, 17, 2, 19, 14, 8, 3, 11, 18, 22], [0, 1, 22, 11, 12, 9, 3, 4, 16, 7, 14, 8, 19, 6, 18], [23, 21, 3, 14, 22, 8, 11, 20, 0, 16, 4, 5, 17], [23, 6, 15, 11, 8, 16, 5, 17, 22], [5, 12, 14, 15, 6, 11, 22, 8, 18, 10, 23, 7, 3], [13, 18, 12, 8, 10, 3, 2], [3, 17, 11, 7, 20, 8, 18, 4, 12, 16, 19], [19, 18, 2, 16, 3, 6, 10, 14, 5, 20]]
def create_adjacency_matrix(adj_list, num_nodes):
    adj_matrix = [[0] * num_nodes for _ in range(num_nodes)]  # Initialize with zeros
    for node, neighbors in enumerate(adj_list):
        for neighbor in neighbors:
            adj_matrix[node][neighbor] = 1
            adj_matrix[neighbor][node] = 1  # Assuming undirected graph
    return adj_matrix
graphs = [graph1, graph2, graph3, graph4, graph5, graph6, graph7, graph8, graph9, graph10]
for i in range(len(graphs)):
    graphs[i] = create_adjacency_matrix(graphs[i], 24)

In [59]:
graph_tensors = [torch.tensor(graph, dtype=torch.float32).unsqueeze(0).to(device) for graph in graphs]
import time
startTime = time.time()
model.eval()  # Set the model to evaluation mode
with torch.no_grad():  # Disable gradient calculation
    for i, graph_tensor in enumerate(graph_tensors):
        # Predict gamma and beta values for each graph
        predicted_gamma_beta = model([graph_tensor, None])  # None for Y since it's not needed during prediction

        # Convert predictions to numpy for easier handling
        predicted_gamma_beta = predicted_gamma_beta.cpu().numpy()

        print(f"Graph {i+1} - Predicted Gamma and Beta values: {predicted_gamma_beta}")
print("ellasped Time: ", time.time()-startTime)

Graph 1 - Predicted Gamma and Beta values: [[ 0.11482838 -0.06553309]]
Graph 2 - Predicted Gamma and Beta values: [[ 0.07576785 -0.0340221 ]]
Graph 3 - Predicted Gamma and Beta values: [[ 0.10506362 -0.03197668]]
Graph 4 - Predicted Gamma and Beta values: [[ 0.12024899 -0.06295986]]
Graph 5 - Predicted Gamma and Beta values: [[ 0.1158793 -0.0235382]]
Graph 6 - Predicted Gamma and Beta values: [[ 0.21388616 -0.11889774]]
Graph 7 - Predicted Gamma and Beta values: [[ 0.11343513 -0.04357992]]
Graph 8 - Predicted Gamma and Beta values: [[ 0.154932  -0.1073247]]
Graph 9 - Predicted Gamma and Beta values: [[ 0.11178534 -0.03763681]]
Graph 10 - Predicted Gamma and Beta values: [[ 0.12161658 -0.10584046]]
ellasped Time:  0.030094385147094727
